# Chapter 2 : Write to Orion

Let's assume you have a NGSI entity ready to be sent to the Orion Context Broker.<br>
Let's continue with the tutorial of the previous chapter.

In [1]:
# our basic NGSI entity from the previous chapter

from datetime import datetime
from pyngsi.ngsi import DataModel

m = DataModel(id="Room1", type="Room")
m.add("dateObserved", datetime.now())
m.add("pressure", 720)
m.add("temperature", 23.0)

m.pprint()

{
  "id": "Room1",
  "type": "Room",
  "dateObserved": {
    "value": "2020-03-27T07:18:39",
    "type": "DateTime"
  },
  "pressure": {
    "value": 720,
    "type": "Integer"
  },
  "temperature": {
    "value": 23.0,
    "type": "Float"
  }
}


## The SinkOrion class

At this point you should have an Orion server up and running.<br>
If not you could run your own local Docker-based server, as described in the Appendix.

**dir()** and **help()** are always useful.

In [3]:
# there are many other sinks mainly to help develop your agent
# use help() for info
# for the moment just focus on Orion
from pyngsi.sink import SinkOrion

# have a look at the init args
help(SinkOrion.__init__)

Help on function __init__ in module pyngsi.sink:

__init__(self, hostname='127.0.0.1', port='1026', baseurl='/v2/entities?options=upsert', service=None, servicepath=None, **kwargs)
    Parameters
    ----------
    hostname : str
        Server hostname
    port : int
        Server port
    baseurl: str
        Server Base URL
    useragent: str
        HTTP User-Agent header sent in the request
    proxy: str
        HTTP Proxy string (i.e http://127.0.0.1:8080)



## Init the Sink

In [4]:
# use our local Orion server
sink = SinkOrion()

2020-03-27 07:18:56.921 | DEBUG    | pyngsi.sink:__init__:209 - init SinkOrion
2020-03-27 07:18:56.922 | DEBUG    | pyngsi.sink:__init__:145 - init SinkHttp
2020-03-27 07:18:56.923 | INFO     | pyngsi.sink:__init__:161 - server url = self.url='http://127.0.0.1:1026/v2/entities?options=upsert'
2020-03-27 07:18:56.924 | INFO     | pyngsi.sink:__init__:162 - User-Agent = useragent='NgsiAgent v1.2.3'
2020-03-27 07:18:56.924 | INFO     | pyngsi.sink:__init__:163 - proxy = None
2020-03-27 07:18:56.925 | INFO     | pyngsi.sink:__init__:232 - No token found. Request Orion without authentication.


## Write entity

In [5]:
# by default it is a silent operation
# it's ok unless an exception is raised
sink.write(m.json())

Congratulations ! You have wrote your first entity to Orion !

## Visualize result

```bash
$ curl http://127.0.0.1:1026/v2/entities
```
[{"id":"Room1","type":"Room","dateObserved":{"type":"DateTime","value":"2020-03-27T07:18:39.00Z","metadata":{}},"pressure":{"type":"Integer","value":720,"metadata":{}},"temperature":{"type":"Float","value":23,"metadata":{}}}]

## Update entity

In [7]:
# update our entity : one degree more !
m.add("dateObserved", datetime.now())
m.add("temperature", 24.0)

# write again
sink.write(m.json())

```bash
$ curl http://127.0.0.1:1026/v2/entities
```
[{"id":"Room1","type":"Room","dateObserved":{"type":"DateTime","value":"2020-03-27T07:27:47.00Z","metadata":{}},"pressure":{"type":"Integer","value":720,"metadata":{}},"temperature":{"type":"Float","value":24,"metadata":{}}}]

## Constuctor args in detail

The default init args target a local installation.<br>
If needed set the **hostname** and **port** according to your Orion server.<p>

**baseurl** SHOULD not be modified.<p>

**service** and **servicepath** have to do with Fiware [multi-tenancy](https://fiware-orion.readthedocs.io/en/master/user/multitenancy/index.html).<br>
When provided, **Fiware-Service** and **Fiware-ServicePath** HTTP Headers are set accordingly.<p>

**useragent** is the **User-Agent** HTTP Header sent to Orion.<br>
It is set by **pyngsi** with its own version. You can override it.<p>

**proxy** allows you to configure a proxy between the agent and Orion, for debugging purpose.<p>

**token** allows you to provide an OAuth2 token for Orion authentication.<br>
Default is no authentication.<p>
Setting a token programmatically is NOT recommended in production mode - for security reasons.<p>
You can safely use one of the two above methods :
- set the environment variable **ORION_TOKEN**
- if using Docker, set a docker secret named **orion_token**

## Status

In [8]:
from pyngsi.sink import SinkOrion

sink = SinkOrion()

# we can ask Orion for its status
status = sink.status()
print(status)

2020-03-27 07:30:28.516 | DEBUG    | pyngsi.sink:__init__:209 - init SinkOrion
2020-03-27 07:30:28.517 | DEBUG    | pyngsi.sink:__init__:145 - init SinkHttp
2020-03-27 07:30:28.517 | INFO     | pyngsi.sink:__init__:161 - server url = self.url='http://127.0.0.1:1026/v2/entities?options=upsert'
2020-03-27 07:30:28.518 | INFO     | pyngsi.sink:__init__:162 - User-Agent = useragent='NgsiAgent v1.2.3'
2020-03-27 07:30:28.519 | INFO     | pyngsi.sink:__init__:163 - proxy = None
2020-03-27 07:30:28.520 | INFO     | pyngsi.sink:__init__:232 - No token found. Request Orion without authentication.
{'orion': {'version': '2.3.0-next', 'uptime': '0 d, 0 h, 14 m, 12 s', 'git_hash': 'c58a5f1667bef42f4097172e026b578e981d67e9', 'compile_time': 'Mon Mar 23 09:45:09 UTC 2020', 'compiled_by': 'root', 'compiled_in': 'de8bb6b5569e', 'release_date': 'Mon Mar 23 09:45:09 UTC 2020', 'doc': 'https://fiware-orion.rtfd.io/'}}
